<a href="https://colab.research.google.com/github/HoseinNekouei/fine_tuning_with_custom_dataset/blob/main/fine_tuning_imdb_review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Load Dataset**

In [1]:
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

In [3]:
!wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz

--2025-02-27 22:43:26--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz.1’

aclImdb_v1.tar.gz.1 100%[===================>]  80.23M  8.04MB/s    in 8.8s    

2025-02-27 22:43:35 (9.08 MB/s) - ‘aclImdb_v1.tar.gz.1’ saved [84125825/84125825]



# **Load the dataset and divide into training and testing set**

In [2]:
from posixpath import join
from pathlib import Path

def read_imdb_split(split_dir):
  split_dir= Path(split_dir)
  texts=[]
  labels=[]

  for label_dir in ['neg', 'pos']:
    for text_file in (split_dir/label_dir).iterdir():

      texts.append(text_file.read_text())
      labels.append(0 if label_dir== 'neg' else 1)

  return texts, labels

train_texts, train_labels= read_imdb_split('/content/aclImdb/train')
# test_texts, test_labels= read_imdb_split('/content/aclImdb/test')

shape=','.join(
    [f'train text length: {len(train_texts)}',
    f'train labels length: {len(train_labels)}',
    # f'test text length: {len(test_texts)}',
    # f'test labels length: {len(test_labels)}'
    ])

print(shape)

train text length: 25000,train labels length: 25000


In [3]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(train_texts, train_labels, test_size= 0.2, stratify= train_labels)

length= ','. join([
    f'X_train length: {len(X_train)}',
    f'X_val length: {len(X_val)}',
    f'y_train length: {len(y_train)}',
    f'y_val length: {len(y_val)}'
])

print(length)

X_train length: 20000,X_val length: 5000,y_train length: 20000,y_val length: 5000


# **Load Tokenizer**

In [4]:
from transformers import AutoTokenizer
check_point= 'FacebookAI/roberta-base'

tokenizer = AutoTokenizer.from_pretrained(check_point)
tokenizer

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


RobertaTokenizerFast(name_or_path='FacebookAI/roberta-base', vocab_size=50265, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	50264: AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False, special=True),
}
)

In [5]:
train_encodings = tokenizer(X_train, padding=True, truncation=True, max_length= 256)
val_encodings= tokenizer(X_val, padding= True, truncation= True, max_length= 256)
# test_encodings= tokenizer(test_texts, padding= True, truncation= True, max_length= 256)

# **Make Dataset and DataLoader**

In [6]:
import torch
torch.cuda.empty_cache()

In [7]:
class IMDBDataset(torch.utils.data.Dataset):
  def __init__(self, encodings, labels):
    self.encodings= encodings
    self.labels= labels

  def __getitem__(self, idx):
    item= {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    item['labels'] = torch.tensor(self.labels[idx])
    return item

  def __len__(self):
    return len(self.labels)


train_set= IMDBDataset(train_encodings, y_train)
val_set= IMDBDataset(val_encodings, y_val)

In [8]:
from torch.utils.data import DataLoader

batch_size = 64

train_loader = DataLoader(train_set, batch_size= batch_size, shuffle=True)
val_loader = DataLoader(val_set, batch_size= batch_size)

In [9]:
from transformers import  AutoModelForSequenceClassification
from torch.optim import AdamW

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model = AutoModelForSequenceClassification.from_pretrained(check_point)
model.to(device)

optim = AdamW(model.parameters(), lr=5e-5)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
# batch =next(iter(train_loader))
# batch['labels'][0]
# input_ids= batch['input_ids'].to(device)
# attention_mask = batch['attention_mask'].to(device)
# labels= batch['labels'].to(device)

# outputs= model(input_ids, attention_mask=attention_mask, labels=labels)
# print(outputs.logits)
# y_pred = torch.argmax(outputs.logits, dim=1)

# print('y_pred',y_pred)
# print('labels',labels)

# acc= y_pred == labels
# result= torch.sum(y_pred==labels).item()
# print(acc)
# print(result)

In [ ]:
import math

epochs= 10
best_loss= float('inf')
patience = 3  # Number of epochs to wait for improvement
epochs_without_improvement = 0
accumulation_steps = 4  # Number of batches to accumulate gradients over

for epoch in range(epochs):
  total_loss, train_loss, val_loss = 0, 0, 0
  total_acc, train_acc, val_acc = 0, 0 , 0

  # Set model to training mode
  model.train()

  optim.zero_grad()   # Clear gradients before starting accumulation
  for i, batch in enumerate(train_loader):
    # Using GPU
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels = batch['labels'].to(device)

    # Forward pass
    outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
    y_pred = torch.argmax(outputs.logits, dim=1)

    # Compute loss
    loss = outputs[0]

    # Backward pass and optimization
    loss.backward()

    # Update model parameters after `accumulation_steps` batches
    if (i + 1) % accumulation_steps == 0:
        optim.step()       # Update model parameters
        optim.zero_grad()  # Clear gradients for the next accumulation cycle

    # Accumulate loss and accuracy
    train_loss += loss.item() * len(batch['input_ids'])
    train_acc += torch.sum(y_pred == labels).item()

    if (i + 1) % 10 == 0:
      print(f' batch: [{i + 1}/{math.floor(len(train_set)/batch_size)}]')

  # Average loss and accuracy for training
  train_loss /= len(train_set)
  train_acc /= len(train_set)


  # Set model to evaluation mode
  model.eval()

  with torch.no_grad():
    for batch in val_loader:

      # Using GPU
      input_ids = batch['input_ids'].to(device)
      attention_mask = batch['attention_mask'].to(device)
      labels = batch['labels'].to(device)

      # Forward pass
      outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
      y_pred= torch.argmax(outputs.logits, dim=1)

      # Compuute loss
      loss= outputs[0]

      # Accumulate loss and accuracy
      val_loss += loss.item() * len(batch['input_ids'])
      val_acc += torch.sum(y_pred == labels).item()

    # Average loss and accuracy for validation
    val_loss /= len(val_set)
    val_acc /= len(val_set)


  # Print training and validation results
  training_result= ','.join([
    f'Epoch:[{epoch + 1}], '
    f'train_Loss:{train_loss:.3f}, '
    f'train_acc:{train_acc:.3f}, '
    f'val_Loss:{val_loss:.3f}, '
    f'val_acc:{val_acc:.3f} '
  ])

  print(training_result)

  # Save the model if validation loss improves
  # Early stopping: Added to prevent overfitting.
  if val_loss < best_loss:
      best_loss = val_loss
      epochs_without_improvement = 0
      print('model saved!')
      torch.save(model, 'imdb_model.pt')
  else:
      epochs_without_improvement += 1
      if epochs_without_improvement >= patience:
          print("Early stopping!")
          break

 batch: [10/312]
 batch: [20/312]
 batch: [30/312]
 batch: [40/312]
 batch: [50/312]
 batch: [60/312]
 batch: [70/312]
 batch: [80/312]
 batch: [90/312]
 batch: [100/312]
 batch: [110/312]
 batch: [120/312]
 batch: [130/312]
 batch: [140/312]
 batch: [150/312]
 batch: [160/312]
 batch: [170/312]
 batch: [180/312]
 batch: [190/312]
 batch: [200/312]
 batch: [210/312]
 batch: [220/312]
 batch: [230/312]
 batch: [240/312]
 batch: [250/312]
 batch: [260/312]
 batch: [270/312]
 batch: [280/312]
 batch: [290/312]
 batch: [300/312]
 batch: [310/312]
Epoch:[1], train_Loss:0.274, train_acc:0.885, val_Loss:0.187, val_acc:0.929 
model saved!
 batch: [10/312]
 batch: [20/312]
 batch: [30/312]
 batch: [40/312]
 batch: [50/312]
 batch: [60/312]
 batch: [70/312]
 batch: [80/312]
 batch: [90/312]
 batch: [100/312]
 batch: [110/312]
 batch: [120/312]
 batch: [130/312]
 batch: [140/312]
 batch: [150/312]
 batch: [160/312]
 batch: [170/312]
 batch: [180/312]
 batch: [190/312]
 batch: [200/312]
 batch: [2

In [ ]:
# with torch.no_grad():
#     outputs = model(input_ids, attention_mask=attention_mask)
#     predictions = torch.argmax(outputs.logits, dim=-1)